In [3]:
import tensorflow as tf

gpu = tf.config.experimental.list_physical_devices("GPU")
print(gpu)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15045364012802652819
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 14282653696
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17007150464670805725
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

# Model 1: CIFAR10

In [5]:
from tensorflow import keras
from tensorflow.keras.layers import Flatten, Dense
import numpy as np
import matplotlib.pyplot as plt

(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train / 255.0
x_test = x_test / 255.0

y_train_ohc = tf.keras.utils.to_categorical(y_train, num_classes=10, dtype='float32')
y_test_ohc = tf.keras.utils.to_categorical(y_test, num_classes=10, dtype='float32')

def get_model_1():
    model = tf.keras.models.Sequential([
        Flatten(input_shape=(32, 32, 3)),
        Dense(3000, activation='relu'),
        Dense(1000, activation='relu'),
        Dense(10, activation='sigmoid')
    ])
    model.compile(optimizer="adam", 
                  loss="categorical_crossentropy", 
                  metrics=["accuracy"])    
    return model


In [6]:
# CPU vs GPU testing
def test_cpu_1(epochs=5):
    with tf.device("/CPU:0"):
        model_cpu = get_model_1()
        model_cpu.fit(x_train, y_train_ohc, epochs=epochs)

def test_gpu_1(epochs=5):
    with tf.device("/GPU:0"):
        model_gpu = get_model_1()
        model_gpu.fit(x_train, y_train_ohc, epochs=epochs)


In [7]:

%timeit -n1 -r1 test_cpu_1(epochs=5)


Epoch 1/5
1563/1563 [==============================] - 55s 35ms/step - loss: 1.8896 - accuracy: 0.3283
Epoch 2/5
1563/1563 [==============================] - 54s 34ms/step - loss: 1.6511 - accuracy: 0.4060
Epoch 3/5
1563/1563 [==============================] - 52s 33ms/step - loss: 1.5745 - accuracy: 0.4358
Epoch 4/5
1563/1563 [==============================] - 51s 33ms/step - loss: 1.5310 - accuracy: 0.4519
Epoch 5/5
1563/1563 [==============================] - 51s 33ms/step - loss: 1.4926 - accuracy: 0.4646
4min 23s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
%timeit -n1 -r1 test_gpu_1(epochs=5)

Epoch 1/5
1563/1563 [==============================] - 8s 4ms/step - loss: 1.8850 - accuracy: 0.3272
Epoch 2/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.6718 - accuracy: 0.4001
Epoch 3/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5891 - accuracy: 0.4289
Epoch 4/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.5327 - accuracy: 0.4490
Epoch 5/5
1563/1563 [==============================] - 6s 4ms/step - loss: 1.4984 - accuracy: 0.4624
33.2 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


# Model 2: Fashion MNIST

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images_scaled = train_images / 255.0
test_images_scaled = test_images / 255.0

def get_model_2(hidden_layers=1):
    # Flatten layer for input
    layers = [keras.layers.Flatten(input_shape=(28, 28))]
    # hidden layers
    for i in range(hidden_layers):
        layers.append(keras.layers.Dense(500, activation='relu'),)
    # output layer    
    layers.append(keras.layers.Dense(10, activation='sigmoid'))
    model = keras.Sequential(layers)
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

def test_cpu_2():
    with tf.device('/CPU:0'):
        cpu_model = get_model_2(hidden_layers=5)
        cpu_model.fit(train_images_scaled, train_labels, epochs=5)

def test_gpu_2():
    with tf.device('/GPU:0'):
        cpu_model = get_model_2(hidden_layers=5)
        cpu_model.fit(train_images_scaled, train_labels, epochs=5)

In [ ]:
%timeit -n1 -r1 test_cpu_2()

In [ ]:
%timeit -n1 -r1 test_gpu_2()